In [ ]:
import os, io
from PIL import Image, ImageDraw, ImageFont
from traitlets import All
import cairosvg
import matplotlib.pyplot as plt

def open_image_any(path, dpi=300):
    ext = os.path.splitext(path)[1].lower()
    if ext == ".svg":
        png_bytes = cairosvg.svg2png(url=path, dpi=dpi)
        return Image.open(io.BytesIO(png_bytes))
    return Image.open(path)


def to_rgb(im, bg_color=(255, 255, 255)):
    """Convert image to RGB, flattening alpha onto a solid background if needed."""
    if im.mode in ("RGBA", "LA"):
        bg = Image.new("RGB", im.size, bg_color)
        bg.paste(im, mask=im.split()[-1])
        return bg
    return im.convert("RGB")

def text_page(size, text, bg=(255, 255, 255), fg=(0, 0, 0)):
    page = Image.new("RGB", size, bg)
    draw = ImageDraw.Draw(page)
    font = ImageFont.load_default()

    x0, y0, x1, y1 = draw.textbbox((0, 0), text, font=font, font_size=120)
    w, h = (x1 - x0), (y1 - y0)

    draw.text(((size[0] - w) // 2, (size[1] - h) // 2), text, fill=fg, font=font, font_size=120)
    return page

def matplotlib_title_page_pil(title, experiment_name, subtitle=None, dpi=300):
    fig = plt.figure(figsize=(8.5, 11), dpi=dpi)
    fig.patch.set_facecolor("white")

    fig.text(0.5, 0.55, title, ha="center", va="center",
             fontsize=24, weight="bold")

    fig.text(0.5, 0.5, experiment_name, ha="center", va="center",
             fontsize=20)

    if subtitle:
        fig.text(0.5, 0.45, subtitle, ha="center", va="center",
                 fontsize=16)

    plt.axis("off")

    buf = io.BytesIO()
    plt.savefig(buf, format="png", dpi=dpi, bbox_inches="tight")
    plt.close(fig)

    buf.seek(0)
    return Image.open(buf).convert("RGB")

def create_and_save_experiment_pdf(experiment_output_dir, experiment_name, model_run):
    print(f"Creating PDF for experiment: {experiment_name}, model run: {model_run}")
    image_dict = {
        "TG Prediction Accuracy": "per_gene_mean_expression.svg",
        "Training R2": "eval_results_pearson_corr.svg",
        "Training and Validation Loss": "train_val_loss_curves.svg",
        "Mean AUROC Pooled": "all_results_auroc_boxplot_pooled.svg",
        "Mean AUPRC Pooled": "all_results_auprc_boxplot_pooled.svg",
        "Mean per-TF AUROC": "all_results_auroc_boxplot_per_tf.svg",
        "Mean per-TF AUPRC": "all_results_auprc_boxplot_per_tf.svg",
        "Method vs GT AUROC Score heatmap": "method_gt_auroc_heatmap_pooled.svg",
        "Method vs GT AUPRC Score heatmap": "method_gt_auroc_heatmap_pooled.svg",
        "Method vs GT per-TF AUROC Score heatmap": "per_tf_auroc_heatmap.svg",
        "Method vs GT per-TF AUPRC Score heatmap": "per_tf_auprc_heatmap.svg",
        
        
    }

    experiment_dir = os.path.join(experiment_output_dir, experiment_name, model_run)

    fig_save_dir = os.path.join(experiment_output_dir, "EXPERIMENT_PDFS")
    os.makedirs(fig_save_dir, exist_ok=True)
    pdf_path = os.path.join(fig_save_dir, f"{experiment_name}_experiment_results.pdf")

    pages = []
    for title, filename in image_dict.items():
        path = os.path.join(experiment_dir, filename)
        if not os.path.isfile(path):
            print(f"Missing: {path}")
            continue

        im = open_image_any(path, dpi=300)
        im_rgb = to_rgb(im)
        pages.append(im_rgb)
        
    # Add title page
    title_page = matplotlib_title_page_pil(
        title=f"Experiment Results",
        experiment_name=experiment_name,
        subtitle=f"{model_run.strip('/')}",
        dpi=300,
    )
    pages.insert(0, title_page)

    # Save multi-page PDF
    pages[0].save(
        pdf_path,
        "PDF",
        resolution=300.0,
        save_all=True,
        append_images=pages[1:],
    )

    print(f"Wrote: {pdf_path}")

experiment_output_dir = "/gpfs/Labs/Uzun/RESULTS/PROJECTS/2024.SINGLE_CELL_GRN_INFERENCE.MOELLER/FIGURES"

experiment_name = "mESC_slow_decay_filter_ten_pct"
model_run = "model_training_001"

create_and_save_experiment_pdf(experiment_output_dir, experiment_name, model_run)

Creating PDF for experiment: mESC_slow_decay_filter_ten_pct, model run: model_training_001/
Wrote: /gpfs/Labs/Uzun/RESULTS/PROJECTS/2024.SINGLE_CELL_GRN_INFERENCE.MOELLER/FIGURES/EXPERIMENT_PDFS/mESC_slow_decay_filter_ten_pct_experiment_results.pdf
